### NN with WoE transformed features

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Normalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import AUC, Precision, Recall
from imblearn.over_sampling import SMOTENC
from scikeras.wrappers import KerasClassifier

In [2]:
train_df = pd.read_csv('train_df_woe.csv',index_col=0)
test_df = pd.read_csv('test_df_woe.csv',index_col=0)

In [3]:
X_train = train_df.drop('stroke',axis=1)
y_train = train_df['stroke']

X_test = test_df.drop('stroke',axis=1)
y_test = test_df['stroke']

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4087 entries, 0 to 4086
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                4087 non-null   float64
 1   hypertension       4087 non-null   float64
 2   heart_disease      4087 non-null   float64
 3   ever_married       4087 non-null   float64
 4   work_type          4087 non-null   float64
 5   avg_glucose_level  4087 non-null   float64
 6   bmi                4087 non-null   float64
 7   smoking_status     4087 non-null   float64
dtypes: float64(8)
memory usage: 287.4 KB


In [7]:
def build_model(input_dim=12, learning_rate=0.0003):
    model = Sequential([
    Dense(32, input_shape=(input_dim,), activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[Recall(name='recall'),AUC(name='auc')] # I think its better to track the models recall instead of accuracy, since we want to catch all strokes
    )
    return model


In [8]:
model = build_model(input_dim=X_train.shape[1])
training = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=[X_test,y_test,],
    verbose=0
)

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_probs))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Accuracy: 0.9471624266144814
ROC AUC: 0.8375707835935109
Confusion Matrix:
 [[968   0]
 [ 54   0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       968
           1       0.00      0.00      0.00        54

    accuracy                           0.95      1022
   macro avg       0.47      0.50      0.49      1022
weighted avg       0.90      0.95      0.92      1022



### Just as without WoE the model only predicts zeroes without smote or class weights

#### Since all features are categorical SMOTE doesn't work well so class weights instead

### Finding the architecture with GridSearch

In [13]:
from sklearn.utils.class_weight import compute_class_weight


class_weights = compute_class_weight(
    'balanced', 
    classes=np.unique(y_train), 
    y=y_train
)
class_weights = dict(enumerate(class_weights))


def build_model(learning_rate=0.0003, 
               dropout_rate=0.2, 
               layer_sizes=[64, 32], 
               use_batch_norm=False,
               activation='relu'):
    model = Sequential()
    model.add(Dense(layer_sizes[0], input_shape=(8,), activation=activation))
    if use_batch_norm:
        model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    for size in layer_sizes[1:]:
        model.add(Dense(size, activation=activation))
        if use_batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[Recall(name='recall'), AUC(name='auc')]
    )
    return model


model = KerasClassifier(
    model=build_model,
    verbose=0,
    class_weight=class_weights  # Default weights (can be overridden in fit)
)


param_grid = {
    'model__learning_rate': [0.0001, 0.0003, 0.001],
    'model__dropout_rate': [0.1, 0.2, 0.3, 0.5],
    'model__layer_sizes': [
        [64, 32],
        [128, 64],
        [64, 64, 32],
        [128, 64, 32]
    ],
    'model__use_batch_norm': [True, False],
    'batch_size': [32, 64, 128],
    'epochs': [20],

}


grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=2
)

# Fit with class weights
grid_result = grid.fit(X_train, y_train)

print(f"\nBest: {grid_result.best_score_:.4f} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"{mean:.4f} ({stdev:.4f}) with: {param}")

# Evaluate best model (unchanged)
best_model = grid_result.best_estimator_.model_
y_pred_probs = best_model.predict(X_test).flatten()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nTest Set Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_probs))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] END batch_size=32, epochs=20, model__dropout_rate=0.1, model__layer_sizes=[64, 32], model__learning_rate=0.0001, model__use_batch_norm=True; total time=   4.2s
[CV] END batch_size=32, epochs=20, model__dropout_rate=0.1, model__layer_sizes=[64, 32], model__learning_rate=0.0001, model__use_batch_norm=True; total time=   4.2s
[CV] END batch_size=32, epochs=20, model__dropout_rate=0.1, model__layer_sizes=[64, 32], model__learning_rate=0.0001, model__use_batch_norm=True; total time=   4.0s
[CV] END batch_size=32, epochs=20, model__dropout_rate=0.1, model__layer_sizes=[64, 32], model__learning_rate=0.0001, model__use_batch_norm=False; total time=   3.3s
[CV] END batch_size=32, epochs=20, model__dropout_rate=0.1, model__layer_sizes=[64, 32], model__learning_rate=0.0001, model__use_batch_norm=False; total time=   3.2s
[CV] END batch_size=32, epochs=20, model__dropout_rate=0.1, model__layer_sizes=[64, 32], model__learning_rate=

In [16]:
grid_result.best_estimator_

KerasClassifier(
	model=<function build_model at 0x000002606C3CA980>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=20
	class_weight={0: 0.5250513874614594, 1: 10.47948717948718}
	model__dropout_rate=0.5
	model__layer_sizes=[128, 64, 32]
	model__learning_rate=0.001
	model__use_batch_norm=True
)

In [14]:
best_model.summary()

Model: "sequential_905"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3149 (Dense)              │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1122        │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2241 (Dropout)          │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3150 (Dense)              │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1123        │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2242 (Dropout)          │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3151 (Dense)              │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1124        │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2243 (Dropout)          │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3152 (Dense)              │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,357 (142.02 KB)

 Trainable params: 11,969 (46.75 KB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 23,940 (93.52 KB)

In [28]:
checkpoint_path = "best_model_GS_woe.keras"
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_auc',   
    mode='max',          
    save_best_only=True,
    verbose=0
)

In [29]:
from sklearn.utils.class_weight import compute_class_weight


training = best_model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    verbose=0,
    callbacks=[checkpoint]
)


In [30]:
best_model.load_weights('best_model_GS_woe.keras')
y_pred_probs = best_model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_probs))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step
Accuracy: 0.6966731898238747
ROC AUC: 0.8238636363636365
Confusion Matrix:
 [[670 298]
 [ 12  42]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.69      0.81       968
           1       0.12      0.78      0.21        54

    accuracy                           0.70      1022
   macro avg       0.55      0.73      0.51      1022
weighted avg       0.94      0.70      0.78      1022



In [31]:
y_pred_probs = best_model.predict(X_test)
y_pred = (y_pred_probs > 0.35).astype(int).flatten()

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_probs))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step
Accuracy: 0.5743639921722113
ROC AUC: 0.8238636363636365
Confusion Matrix:
 [[540 428]
 [  7  47]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.56      0.71       968
           1       0.10      0.87      0.18        54

    accuracy                           0.57      1022
   macro avg       0.54      0.71      0.45      1022
weighted avg       0.94      0.57      0.68      1022



Final Model's with WoE most important metrics
1. AUC = 0.8238
2. TPR = 0.87
3. Macro avg precision = 0.54
4. Macro avg recall = 0.71
5. Macro avg f1 = 0.45

The model performed basicly the same as the one without woe transformed features